# VacationPy

In [38]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import g_key
from config import weather_api_key
from citipy import citipy
from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
city_weather_data = pd.read_csv("../WeatherPy/Clean City Data.csv")
city_weather_df = pd.DataFrame(city_weather_data)
city_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hemisphere
0,Mayo,38.89,-76.51,77.00,94,90,5.82,US,1601231325,Southern Hemisphere
1,Tazovsky,67.47,78.70,39.02,84,100,30.87,RU,1601231325,Northern Hemisphere
2,Hermanus,-34.42,19.23,46.00,86,22,3.31,ZA,1601231295,Northern Hemisphere
3,College,64.86,-147.80,43.00,75,75,5.82,US,1601231326,Southern Hemisphere
4,Nizwá,22.93,57.53,87.39,18,9,6.62,OM,1601231327,Northern Hemisphere


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
# Createa  dataframe with latitude and longitude
locations_df = city_weather_df[["Lat", "Lng"]]

# Extract a list of tuples from a dataframe
records = locations_df.to_records(index=False)
locations = list(records)

gmaps.configure(g_key)
fig=gmaps.figure(map_type="HYBRID")
fig.add_layer(gmaps.heatmap_layer(locations, weights=city_weather_df["Humidity"]))

fig = gmaps.figure(layout={
        'width': '800px',
        'height': '500px',
        'padding': '3px',
        'border': '1px solid black'
})

fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='3px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [41]:
# Ideal cities criteria
# Temp: Between 80 and 95
# Humidity: Less Than 65
# Cloudiness: Less than 30%
# Wind Speed: Between 5 and 15

ideal_cities_df = city_weather_df
ideal_cities_df = ideal_cities_df.loc[((ideal_cities_df["Max Temp"] < 95) & (ideal_cities_df["Max Temp"] > 79)) & \
                                      (ideal_cities_df["Humidity"] <= 65) & (ideal_cities_df["Cloudiness"] <= 30) & \
                                      ((ideal_cities_df["Wind Speed"] <= 15) & (ideal_cities_df["Wind Speed"] >= 5))]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna(axis=0, how="any")
ideal_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hemisphere
4,Nizwá,22.93,57.53,87.39,18,9,6.62,OM,1601231327,Northern Hemisphere
86,Awjilah,29.11,21.29,91.54,24,0,6.78,LY,1601231367,Northern Hemisphere
92,Nicosia,35.17,33.37,80.01,61,0,6.93,CY,1601231368,Northern Hemisphere
101,Millbrook,32.48,-86.36,86.00,62,1,5.82,US,1601231371,Southern Hemisphere
110,Uauá,-9.84,-39.48,94.48,24,15,9.42,BR,1601231373,Southern Hemisphere


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels within 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
hotel_df = ideal_cities_df
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hemisphere,Hotel Name
4,Nizwá,22.93,57.53,87.39,18,9,6.62,OM,1601231327,Northern Hemisphere,
86,Awjilah,29.11,21.29,91.54,24,0,6.78,LY,1601231367,Northern Hemisphere,
92,Nicosia,35.17,33.37,80.01,61,0,6.93,CY,1601231368,Northern Hemisphere,
101,Millbrook,32.48,-86.36,86.00,62,1,5.82,US,1601231371,Southern Hemisphere,
110,Uauá,-9.84,-39.48,94.48,24,15,9.42,BR,1601231373,Southern Hemisphere,
117,Las Vegas,36.17,-115.14,89.60,16,1,7.34,US,1601230700,Southern Hemisphere,
118,Encruzilhada,-15.53,-40.91,87.64,25,0,9.01,BR,1601231375,Southern Hemisphere,
171,Makakilo City,21.35,-158.09,80.60,61,1,5.82,US,1601231389,Southern Hemisphere,
184,Nānākuli,21.39,-158.15,80.60,61,1,5.82,US,1601231393,Southern Hemisphere,
185,Panabá,21.28,-88.27,93.99,50,26,8.05,MX,1601231393,Southern Hemisphere,


In [48]:
hotel_df.shape

(32, 11)

In [71]:
hotel_list = []


distance = '5000'
act_type = 'Hotel'
keyword = 'Hotel'

#API structure from API documentation
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?
#location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"

hotel_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=" 
    location = f'{latitude},{longitude}'
    hotels = (hotel_url + location + '&radius=' + distance + '&type=' + act_type + '&keyword=' + keyword + '&key=' + g_key)
   
    response = requests.get(hotels)
    
    response = response.json()
    
    try:
        hotel_list.append(response['results'][0]['name'])        
    except:
        hotel_list.append(np.nan)   
        

hotel_list

['Antique inn',
 nan,
 'Hilton Nicosia',
 'Home2 Suites By Hilton Prattville',
 'Hotel Vaza Barris',
 'The STRAT Hotel, Casino & SkyPod',
 'Pousada Dona Vitória',
 "Marriott's Ko Olina Beach Club",
 nan,
 nan,
 'Pousada Encanto Abrolhos',
 'Hotel Los Caracoles',
 'Fairfield Inn & Suites by Marriott Sacramento Elk Grove',
 'Sur Plaza Hotel',
 'Hotel Vila Gale Touros',
 'Royal Amazonia',
 'Station Total Filingué',
 'La Quinta Inn & Suites by Wyndham Stephenville',
 'HOTEL R. SOARES',
 'Assiut hotels Armed Forces',
 'Jalu Hotel',
 'Sofitel Legend Old Cataract Aswan',
 'Hotel Senac Ilha do Boi',
 'فندق مزدة الكبير',
 'Jahan Serwer',
 'Holiday Inn Express & Suites Grants - Milan',
 'Maui Beach Hotel',
 'Radisson Blu Hotel & Conference Center, Niamey',
 'Hotel Pousada Quinta do Sul',
 'Matizze Hotel',
 'Pousada,Restaurante e Ponto Chic',
 'Shahin Tower Hotel']

In [75]:
hotel_df["Hotel Name"] = hotel_list
hotel_df.dropna(axis = 0, how="any", inplace=True)
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hemisphere,Hotel Name,Hotel
4,Nizwá,22.93,57.53,87.39,18,9,6.62,OM,1601231327,Northern Hemisphere,Antique inn,Antique inn
92,Nicosia,35.17,33.37,80.01,61,0,6.93,CY,1601231368,Northern Hemisphere,Hilton Nicosia,Hilton Nicosia
101,Millbrook,32.48,-86.36,86.00,62,1,5.82,US,1601231371,Southern Hemisphere,Home2 Suites By Hilton Prattville,Tru by Hilton Prattville
110,Uauá,-9.84,-39.48,94.48,24,15,9.42,BR,1601231373,Southern Hemisphere,Hotel Vaza Barris,Hotel Vaza Barris
117,Las Vegas,36.17,-115.14,89.60,16,1,7.34,US,1601230700,Southern Hemisphere,"The STRAT Hotel, Casino & SkyPod","Treasure Island - TI Hotel & Casino, a Radisso..."


In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [79]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='3px', width='800px'))